# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [39]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [40]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/brendanturner/Desktop/UdacityProjects/DataEngineer/CassandraModelingProject


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [41]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [42]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [43]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [44]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [45]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Used SessionID as partition key so we still return all the itemInSession.  itemInSession was used as cluster column to allow for filtering and to make unique.

In [46]:
try:
    session.execute("""create table if not exists artist_song_length 
                    (sessionId int, itemInSession int, artist text, song text, length decimal, primary key (sessionId, itemInSession))""")

except Exception as e:
    print(e)
                    

In [47]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "insert into sparkify.artist_song_length (sessionID, itemInSession, artist, song, length) "
        query = query + "values (%s, %s, %s, %s, %s)"

        session.execute(query, (line[0], line[9],float(line[5]), int(line[8]), int(line[3])))

In [48]:
try:
    rows = session.execute("select artist, song, length from artist_song_length where sessionid = 338 and iteminsession = 4")

except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Used SessionId and UserID as partition keys for filtering and partitioning, and iteminsession as clusting column which sorts the data by this column.

In [49]:
try:
    session.execute("""create table if not exists artist_song_user
                    (sessionid int, userid int, iteminsession int, artist text, song text, firstname text, lastname text, primary key ((sessionid, userid), iteminsession))""")

except Exception as e:
    print(e)

ile = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "insert into sparkify.artist_song_user (sessionid, userid, iteminsession, artist, song, firstname, lastname) "
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9],int(line[8]), int(line[3]), line[1], line[4], int(line[10])))

try:
    rows = session.execute("select artist, song, firstname, lastname from artist_song_user where sessionid = 182 and userid = 10")

except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)  

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Used song as partition key for partitioning, and userid as clusting column to make the primary key unique.

In [50]:
try:
    session.execute("""create table if not exists users
                    (song text, userid int, firstname text, lastname text, primary key (song, userid))""")

except Exception as e:
    print(e)

ile = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into sparkify.users (song, userid, firstname, lastname)"
        query = query + "values (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

try:
    rows = session.execute("select firstname, lastname from users where song = 'All Hands Against His Own'")

except Exception as e:
    print(e)

for row in rows:
    print (row.firstname, row.lastname)  
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [51]:
try:
    session.execute("drop table artist_song_length")
    session.execute("drop table users")
    session.execute("drop table artist_song_user")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [52]:
session.shutdown()
cluster.shutdown()